# Esercizio 5

Prendere in input un file in formato `GTF` (Gene Transfer Format), che annota un set di geni su una genomica di riferimento, insieme al file `FASTA` della genomica di riferimento e produrre:

- le sequenze dei trascritti oppure le sequenze delle coding sequences (CDS) per i geni annotati in formato `FASTA`, a seconda della scelta dell'utente
- il set degli HUGO NAMES dei geni per cui è stata prodotta una sequenza (trascritto oppure CDS) al punto precedente

L'*header* `FASTA` di ogni sequenza prodotta deve contenere:

- lo HUGO name del gene di riferimento
- l’identificatore del trascritto di riferimento
- la lunghezza della sequenza prodotta
- il tipo di sequenza (trascritto o CDS)
- lo strand del gene
    
Esempio di *header* per un trascritto:
         
    >ARHGAP4; U52112.4-003; len=3235 type=transcript; strand=-

Esempio di *header* per una CDS:

    >AVPR2; U52112.2-003; len=642; type=cds; strand=+
   
***

Parametri in input:

- file in formato `GTF`
- file della genomica di riferimento in formato `FASTA`
- *feature* della sequenza da ricostruire: `exon` se si vogliono ricostruire i trascritti o `CDS` se si vogliono ricostruire le coding sequences

***

Requisiti:

- deve essere definita una funzione `format_fasta()` che prenda come argomenti un header `FASTA` e una sequenza, e restituisca la sequenza in formato FASTA separata in righe di 80 caratteri.

- deve essere definita una funzione `reverse_complement()` che prenda come argomento una sequenza nucleotidica e ne restituisca il reverse&complement.

- deve essere definita una funzione `compose_feature()` che prenda come argomenti una lista di features come tuple *(start, end)*) di *features*, la genomica di riferimento, lo strand del gene di riferimento ed effettui la concatenazione delle sequenze delle *features*, eventualmente operando il reverse&complement se lo strand è `-`.

**NOTA BENE**: gli attributi del nono campo del file `GTF` non sono ad ordine fisso all'interno del campo. Per estrarre quindi un determinato attributo si deve usare un'espressione regolare e non il metodo `split()`.

***

## Soluzione

Importare il modulo `re` per usare le espressioni regolari.

In [ ]:
import re

### Definizione della funzione `format_fasta()`

La funzione prende come argomento una stringa contenente un *header* `FASTA` e una sequenza (nucleotidica o di proteina) e restituisce la sequenza in formato `FASTA` separata in righe di 80 caratteri.

In [ ]:
def format_fasta(header, sequence):
    return header + '\n' + '\n'.join(re.findall('\w{1,80}', sequence))

**NOTA BENE**: supporre che l'*header* in input alla funzione abbia già il simbolo `>` all'inizio, ma non il simbolo `\n` alla fine.

### Definizione della funzione `reverse_complement()`

La funzione prende come argomento una stringa contenente una sequenza nucleotidica e restituisce la versione *reverse and complement* della sequenza.

In [ ]:
def reverse_complement(sequence):
    sequence = sequence.lower()
    sequence = sequence[::-1]
    complement = {'a':'t', 't':'a', 'c':'g', 'g':'c'}
    return ''.join([complement[c] for c in sequence])

reverse_complement('aaattt')

**NOTA BENE**: fare in modo che la funzione sia indipendente dal caso della sequenza in input (maiuscolo o minuscolo).

### Definizione della funzione `compose_feature()`

La funzione prende come argomenti una lista di features come tuple *(start, end)*, la genomica di riferimento, lo strand del gene, ed effettua la concatenazione delle sequenze delle *features*, ed eventualmente il reverse&complement se lo strand del gene è `-`.

In [ ]:
def compose_feature(feature_list, reference_sequence, strand):
    reconstructed_sequence = ''.join(reference_sequence[f[0]-1:f[1]] for f in sorted(feature_list))
    
    if strand == '-':
        reconstructed_sequence = reverse_complement(reconstructed_sequence)
        
    return reconstructed_sequence

**NOTA BENE**: concatenare le sequenze delle *features* sempre per coordinate crescenti.

### Parametri in input

In [ ]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'
feature_name = 'CDS'

### Lettura del file `FASTA` della genomica di riferimento

Lettura del file della genomica di riferimento nella lista di righe `reference_file_rows`

In [ ]:
with open(reference_file_name, 'r') as input_file:
    reference_file_rows = input_file.readlines()

In [ ]:
reference_file_rows

Determinazione della sequenza di riferimento in un'unica stringa.

In [ ]:
genomic_reference = ''.join(reference_file_rows[1:]).rstrip().replace('\n', '')

In [ ]:
genomic_reference

Lettura del file della genomica di riferimento nella stringa `reference_file_string`

In [ ]:
with open(reference_file_name, 'r') as input_file:
    reference_file_string = input_file.read()

reference_file_string

Determinazione della sequenza di riferimento in un'unica stringa.

In [ ]:
genomic_reference2 = ''.join(re.findall('\n(\w+)', reference_file_string))

In [ ]:
genomic_reference2

In [ ]:
genomic_reference2 == genomic_reference

### Lettura dei *record* del file `GTF`

Lettura dei *record* del file `GTF` nella lista di righe `gtf_file_rows`

In [ ]:
with open(gtf_file_name, 'r') as input_file:
    gtf_file_rows = input_file.readlines()

In [ ]:
gtf_file_rows

### Filtraggio dei *record* `GTF` che occorrono per ricostruire le sequenze del tipo scelto

Eliminare dalla lista `gtf_file_rows` i *record* `GTF` che non corrispondono al tipo di *feature* che compone la sequenza che si è scelto di ricostruire, cioé per i quali il terzo campo non è uguale al valore della variabile `feature_name` (`exon` se si è scelto di ricostruire i trascritti full-length e `CDS` se si è scelto di ricostruire le coding sequences).

In [ ]:
gtf_file_rows = [row for row in gtf_file_rows if row.rstrip().split()[2] == feature_name]

In [ ]:
gtf_file_rows

### Costruzione del dizionario degli *strand* e del set dei geni annotati

A partire dalla lista precedente, costruire:
- il dizionario degli *strand*:
    - *chiave*: HUGO name del gene
    - *valore*: strand del gene (`+` o `-`)
    
- il set dei geni annotati relativamente al tipo di sequenza che si vuole ricostruire

**NOTA BENE**: il valore dello *strand* (settimo campo del *record* `GTF`) è costante per un determinato gene.

Inizializzazione del dizionario vuoto.

In [ ]:
strand_dict = {}

Attraversare la lista dei record di tipo uguale a `feature_name` e riempire il dizionario.

In [ ]:
for row in gtf_file_rows:
    strand = row.rstrip().split('\t')[6]
    #hugo_name = re.search('[\w\s;]+gene_id\s"(\w+)', row.rstrip().split('\t')[8]).group(1)
    hugo_name = re.search('gene_id\s"(\w+)";', row).group(1)
    strand_dict[hugo_name] = strand

In [ ]:
strand_dict

Estrarre dal dizionario il set dei geni annotati.

In [ ]:
gene_set = set(strand_dict)

In [ ]:
gene_set

### Ricostruzione delle sequenze

Costruire:

- il dizionario degli ID dei trascritti:
    - *chiave*: HUGO name del gene
    - *valore*: set dei `transcript_id` coinvolti in record di tipo `exon` (se si vogliono ricostruire i trascritti) oppure in record di tipo `CDS` (se si vogliono ricostruire le coding sequence)
    
    
- il dizionario delle composizioni in features:
    - *chiave*: identificatore del trascritto
    - *valore*: lista delle tuple *(start, end)* delle features (records) che compongono la sequenza da ricostruire (trascritto oppure coding sequence) per il trascritto

Inizializzare i dizionari vuoti.

In [ ]:
id_dict = {}
composition_dict = {}

Attraversare la lista `gtf_file_rows` e riempire i due dizionari.

In [ ]:
for row in gtf_file_rows:
    hugo_name = re.search('gene_id\s"(\w+)";', row).group(1)
    transcript_id = re.search('transcript_id\s"([^"]+)";', row).group(1)
    
    feature_start = row.rstrip().split('\t')[3]
    feature_end = row.rstrip().split('\t')[4]
    
    id_dict_value = id_dict.get(hugo_name, set())
    id_dict_value.add(transcript_id)
    id_dict.update([(hugo_name, id_dict_value)])
    
    composition_dict_value = composition_dict.get(transcript_id, list())
    composition_dict_value.append((int(feature_start), int(feature_end)))
    composition_dict.update([(transcript_id, composition_dict_value)])

**NOTA BENE**: un'espressione regolare simile a quella usata per estrarre lo HUGO NAME, cioé `'transcript_id\s+"(\w+)";'` non può funzionare per estrarre dal *record* l'ID del trascritto in quanto in tale ID è presente anche il simbolo di punto `.` che non fa parte della classe dei simboli di parola rappresentata da `\w`. Quindi è meglio usare l'espressione regolare `'transcript_id\s+"([^"]+)";'`.

In [ ]:
id_dict

In [ ]:
composition_dict

A partire dai dizionari precedenti, costruire la lista di tuple *(header, sequenza)* in cui il primo elemento è l'*header* `FASTA` e il secondo elemento è la sequenza ricostruita.

L'*header* deve essere del tipo:

    >ARHGAP4; U52112.4-003; len=3235; type=transcript; strand=-
    
se si è scelto di ricostruire i trascritti full-length, e:

    >ARHGAP4; U52112.4-005; len=642; type=cds; strand=-
    
se si è scelto di ricostruire le coding sequences (CDS).

In [ ]:
sequence_fasta_list = []

sequence_type = {'exon' : 'transcript', 'CDS' : 'cds'}

for hugo_name in id_dict:
    for transcript_id in id_dict[hugo_name]:
        r_sequence = compose_feature(composition_dict[transcript_id], genomic_reference, strand_dict[hugo_name])
        header = '>' + hugo_name + '; ' + transcript_id + '; len=' + str(len(r_sequence)) + '; type=' + sequence_type[feature_name] + '; strand=' + strand_dict[hugo_name]
        sequence_fasta_list.append((header, r_sequence))

In [ ]:
sequence_fasta_list

Trasformare la lista di tuple in una lista di sequenze in formato `FASTA`.

In [ ]:
sequence_fasta_list = [format_fasta(t[0], t[1]) for t in sequence_fasta_list]

In [ ]:
for seq in sequence_fasta_list:
    print(seq)